<h3> Load Libraries </h3>

In [1]:
import glob
import os
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import refet
import pvlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

<h3> Global Variables </h3>

In [ ]:
base_path = "/Users/saraawad/Desktop/Datasets/Google/"

In [2]:
class Helpers:
    def __init__(self):
        print("Helper")

    def drop_nan_columns(df):
        """Drops the columns having all theirs rows as Nans"""
        columns_to_exclude = ["Date", "Day", "Year", "Month", "Timestamp start"
                              , "Time", "TIMESTAMP", "Tier", "TIMESTAMP_START", "TIMESTAMP_END", "Day Status"]
        columns = df.columns
        for i in range(len(columns)):
            col = columns[i]
            if col in columns_to_exclude:
                continue
            nan_sum_col = df[col].isnull().sum()
            if nan_sum_col == len(df):
                df.drop(col, axis=1, inplace=True)
        return df
    
    
    def convert_missing_values_nan(df):
        '''This function will convert -9999 to NaN'''
        df = df.replace(-9999.000000, np.NaN)
        return df

    def get_all_matching_columns(df, keyword):
        return df.filter(like=keyword).columns

    def generate_lags(df, column, lags_count): 
        for i in range(lags_count):
            lag_name = column + "-" + str(i + 1)
            df[lag_name] = df[column].shift(i + 1)
            for j in range(i):
                df.loc[str(j+1), lag_name] = np.nan
        return df

    def add_LE_conversion_rate(df, col):
        conversion_rate = 28.94
        new_col = col + "(mm)"
        df[new_col] = df[col] / conversion_rate
        return df

    def read_sites_data():
        file_path = os.path.join(base_path, "filtered_sites_all.xlsx")
        df = pd.read_excel(file_path)
        df.head()
        return df

    def export_data(df, file_path):
        export_path = os.path.join(base_path, file_path + ".csv")
        export_csv = df.to_csv(export_path, index=None, header=True)

    def load_data(file_path):
        df = pd.read_csv(file_path + ".csv", delimiter=',')
        return df
    
    def list_to_df(list_to_convert):
        '''This function will convert the provided list into a dataframe'''
        df = pd.concat(list_to_convert, sort=True)
        return df
    
    def get_files_directory(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
        listOfFile = os.listdir(dirName)
        allFiles = list()
        # Iterate over all the entries
        for entry in listOfFile:
            # Create full path
            if entry.endswith(".xlsx") or entry.endswith(".icloud") or entry.endswith(".DS_Store"):
                continue
            fullPath = os.path.join(dirName, entry)
            # If entry is a directory then get the list of files in this directory 
            if os.path.isdir(fullPath):
                allFiles = allFiles + Helpers.get_files_directory(fullPath)
            else:
                allFiles.append(fullPath)

        return allFiles

    def concat_dataframe_from_files(files):
        values = []
        for i in range(len(files)):
            file_path = files[i]
            head, file_name = os.path.split(file_path)
            #Get only the sheets having the variables
            if file_name.endswith(".csv"):
                print("file name", file_name)
                df = pd.read_csv(file_path, delimiter=',')
                site_id = file_name.split("_")[0]
                df["Site Id"] = site_id
                values.append(df)
        return Helpers.list_to_df(values)   
    
    def generate_joint_dataframe(dirName):
        files = Helpers.get_files_directory(dirName)
        df = Helpers.concat_dataframe_from_files(files)
        return df
    
    def string_to_date(df, column, dateFormat="%Y%m%d%H%M"):
        df[column] = pd.to_datetime(df[column], format=dateFormat)
        

In [3]:
class AggregateData:
    def __init__(self, path):
        print("Aggregate Data Initializer")
        self.path = path
        
    def prepare_data(self):
        folder_path = os.path.join(base_path, self.path)
        df = Helpers.generate_joint_dataframe(folder_path)
        return df

#Change path in here to read reference evapotranspiration ET from
path = "EEflux/ETr/"
am = AggregateData(path)
df = am.prepare_data()
df.head()


df = df[["Date", "Site Id", "Cloud", "Image Id", "ETo", "Mean ETo"]]
#Save data in path
Helpers.export_data(df, "EEflux_refET_sites")

Aggregate Data Initializer
file name US-Shd_EEflux_ETo.csv
file name US-KFS_EEflux_ET0 2.csv
file name US-FR2_EEflux_ET0.csv
file name US-Wgr_EEflux_ETo 2.csv
file name US-Wlr_EEflux_ETo.csv
file name US-Skr_EEflux_ETo.csv
file name US-Myb_EEflux_ET0.csv
file name US-WBW_EEflux_ETo.csv
file name US-KFS_EEflux_ETo.csv
file name US-AR2_EEflux_ETo.csv
file name US-SP2_EEflux_ETo.csv
file name US-Me6_EEflux_ETo.csv
file name US-A32_EEflux_ETo.csv
file name US-Var_EEflux_ET0.csv
file name US-Kon_EEflux_ET0.csv
file name US-Goo_EEflux_ETo.csv
file name US-ARM_EEflux_ETo.csv
file name US-Pon_EEflux_ETo.csv
file name US-Snd_EEflux_ET0.csv
file name US-Bi2_EEflux_ETo.csv
file name US-Wgr_EEflux_ETo.csv
file name US-Twt_EEflux_ETo.csv
file name US-AR1_EEflux_ET0.csv
file name US-SO2_EEflux_ETo.csv
file name US-Tw3_EEflux_ETo.csv
file name US-Me2_EEflux_ET0.csv
file name US-Tw1_EEflux_ETo.csv
file name US-A74_EEflux_ETo.csv
file name US-MMS_EEflux_ET0.csv
file name US-AR2_EEflux_ETo 2.csv
file na

In [7]:
class AmerifluxHourlyRefET:
    def __init__(self, path, eeflux_path):
        print("AmerifluxHourlyRefET Initializer")
        self.path = path
        self.eeflux_path = eeflux_path
        
        
    def read_data_sites_info(self, site_id):
        file_path = os.path.join(base_path, "filtered_sites_all.xlsx")
        sites_df = pd.read_excel(file_path)
        site_df = sites_df[sites_df["Site Id"] == site_id]
        return site_df
        
    def set_coordinates(self, df, site_id):
        site_df = self.read_data_sites_info(site_id)
        elevation = site_df.iloc[0,3]
        latitude = site_df.iloc[0,1]
        longitude = site_df.iloc[0,2]
        df["elevation"] = elevation
        df["latitude"] = latitude
        df["longitude"] = longitude
        return df
        
    def get_day_of_year(self, df):
        date_time_index = df.columns.get_loc("TIMESTAMP_END")
        day_of_year_list = []
        times = []
        hours = []
        for i in range(len(df)):
            date_str = str(df.iloc[i, date_time_index])
            date = datetime.datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
            day_of_year = date.timetuple().tm_yday
            day_of_year_list.append(day_of_year)

            date_time = date.time()
            hour = int(str(date_time).split(":")[0])
            
            times.append(date_time)
            hours.append(hour)

        df["doy"] = day_of_year_list
        df["time"] = times
        df["hour"] = hours
        
    def add_LE_conversion_rate(self, df, col):
        conversion_rate = 28.94
        new_col = col + "(mm)"
        df[new_col] = df[col] / conversion_rate
        return df
    
    def add_LE_converstion_to_LE(self, df):
        le_columns = list(Helpers.get_all_matching_columns(df, "LE_"))
        columns_list = []
        if "LE" in df.columns:
            columns_list.append("LE")
        columns_list.extend(le_columns)

        for k in range(len(columns_list)):
            col = columns_list[k]
            df = Helpers.add_LE_conversion_rate(df, col)
        return df
        
    def set_ref_ET_values(self, df):
        df["ea"] = ""
        solar_rad_col = list(Helpers.get_all_matching_columns(df, "SW_IN"))
        if len(solar_rad_col) > 0:
            solar_rad_col = solar_rad_col[0]
            df["rs"] = df[solar_rad_col] * 0.0036
        else:
            print("No rs")
            df["rs"] = pvlib.irradiance.get_extra_radiation(df["doy"], 0.0820 ,'asce')
        ws_col = list(Helpers.get_all_matching_columns(df, "WS"))[0]
        df["uz"] = np.where(df[ws_col].isnull(), 2, df[ws_col])
        if "TA" in df.columns:
            ta_col = "TA"
        else:
            ta_col = list(Helpers.get_all_matching_columns(df, "TA_"))[0]
        df["tmean"] = df[ta_col]
        rh_col = list(Helpers.get_all_matching_columns(df, "RH"))
        
        if len(rh_col) > 0 :
            rh_col = rh_col[0]
            df["rh"] = df[rh_col]
        else:
            print("no rh")
            return
    
        print("ws:", ws_col, "rs:", solar_rad_col, "tmean:", ta_col, "rh:", rh_col)
        zw = 2
        df["zw"] = zw
        df["etr"] = ""
        
        elev_index = df.columns.get_loc("elevation")
        lat_index = df.columns.get_loc("latitude")
        lon_index = df.columns.get_loc("longitude")
        ea_index = df.columns.get_loc("ea")
        uz_index = df.columns.get_loc("uz")
        rs_index = df.columns.get_loc("rs")
        rh_index = df.columns.get_loc("rh")
        doy_index = df.columns.get_loc("doy")
        tmean_index = df.columns.get_loc("tmean")
        time_index = df.columns.get_loc("time")
        hour_index = df.columns.get_loc("hour")
        etr_index = df.columns.get_loc("etr")
        
        es = 0.6108 * np.exp(17.27 * df["tmean"] / (df["tmean"] + 237.3))
        ea = df["rh"] / 100 * es
        df["ea"] = ea

        etr = refet.Hourly(tmean= df["tmean"], ea=df["ea"] , rs=df["rs"],
                           uz=df["uz"], zw=df["zw"], elev=df["elevation"],
                           lat=df["latitude"], lon=df["longitude"], doy=df["doy"],
                           time=df["hour"], method='asce').etr()
        df["etr"] = etr

        
    def resample_date_todays(self, df, dictionary):
        time_stamp = []
        df["Date"] = ""
        for i in range(df.shape[0]):
            index_date_column = df.columns.get_loc("TIMESTAMP_START")
            date = datetime.datetime.strptime(str(df.iloc[i, index_date_column]),
                                              "%Y-%m-%d %H:%M:%S").strftime('%m/%d/%y')
            time_stamp.append(date)
        df["Date"] = time_stamp
        df = df.groupby(['Date'], as_index=False).agg(dictionary)
        print("daily ameriflux df", df.shape)
        return df


    def read_eeflux(self, eeflux_path, site_id):
        file_path = os.path.join(base_path, eeflux_path)
        df = pd.read_csv(file_path, delimiter=',')
        df = df[df["Site Id"] == site_id]
        df["Date"] = pd.to_datetime(df["Date"])
        return df

    def merge_eeflux_ameriflux_refET(self, eeflux_df, am_df):
        df = pd.merge(eeflux_df, am_df, left_on=["Date"], 
                         right_on=["Date"], how="inner")
        print("merged df", df.shape)
        return df
    
    def generate_dictionary(self, df):
        sum_columns = []
        if "LE" in df.columns:
            sum_columns.append("LE")
        sum_columns.extend(list(Helpers.get_all_matching_columns(df, "LE_")))
        sum_columns.extend(list(Helpers.get_all_matching_columns(df, "mm")))
        
        dictionary_input = {
        "rh": np.mean,
        "tmean": np.mean,
        "rs": np.mean,
        "ea": np.mean,
        "uz": np.mean,
        "etr": sum
        }

        dictionary_output = {
            i : sum for i in sum_columns
        }
        result_dictionary = {**dictionary_input , **dictionary_output}
        return result_dictionary
    
    def prepare_data(self):
        folder_path = os.path.join(base_path, self.path)
        files = Helpers.get_files_directory(folder_path)
        print("files count:", len(files))
        all_merged_dfs = []
        for i in range(len(files)):
            file_path = files[i]
            head, file_name = os.path.split(file_path)
            if file_name.endswith(".csv"):
            #Skip first 2 lines, meaningless data
                df = pd.read_csv(file_path, delimiter=',', skiprows=2)
                site_id = file_name.split("_")[1]
                print("Site Id:", site_id)
                df["Site Id"] = site_id
                df = df.replace(-9999.000000, np.NaN)
                df = Helpers.drop_nan_columns(df)
                Helpers.string_to_date(df, "TIMESTAMP_START")
                Helpers.string_to_date(df, "TIMESTAMP_END")
                self.get_day_of_year(df)
                df = self.add_LE_converstion_to_LE(df)
                df = self.set_coordinates(df, site_id)
                self.set_ref_ET_values(df)
                if "rh" not in df.columns:
                    continue
                dictionary = self.generate_dictionary(df)
                df = self.resample_date_todays(df, dictionary)
                df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%y")
                eeflux_df = self.read_eeflux(self.eeflux_path, site_id)
                merged_df = self.merge_eeflux_ameriflux_refET(eeflux_df, df)
                merged_df["ETo EEflux"] = np.where(merged_df["ETo"].isnull(), merged_df["Mean ETo"],
                                 merged_df["ETo"])
                ameriflux_eeflux_path = "Ameriflux/Generated/v1/Ameriflux EEflux RefET/" + site_id + "_Ameriflux_EEflux_refET"
                Helpers.export_data(merged_df, ameriflux_eeflux_path)
                all_merged_dfs.append(merged_df)
                print("-----------------------------------------------------------------")
        return all_merged_dfs
    
#Change path in here to read Ameriflux Hourly data from
path = "Ameriflux/AmeriFlux Hourly/"
#Change export path for all eeflux reference evapotranspiration 
eeflux_path = "EEflux_refET_sites.csv"
am = AmerifluxHourlyRefET(path, eeflux_path)
all_merged_list = am.prepare_data()
#Change path in here for the joint data
all_df = Helpers.list_to_df(all_merged_list)
ameriflux_eeflux_path = "Ameriflux/Generated/v1/Ameriflux_EEflux_refET"
Helpers.export_data(all_df, ameriflux_eeflux_path)

AmerifluxHourlyRefET Initializer
files count: 32
Site Id: US-Tw1
ws: WS rs: SW_IN tmean: TA rh: RH
daily ameriflux df (3111, 13)
merged df (643, 18)
-----------------------------------------------------------------
Site Id: US-Skr
ws: WS rs: SW_IN tmean: TA rh: RH
daily ameriflux df (2922, 9)
merged df (714, 14)
-----------------------------------------------------------------
Site Id: US-Snd
ws: WS rs: SW_IN tmean: TA rh: RH
daily ameriflux df (2922, 9)
merged df (342, 14)
-----------------------------------------------------------------
Site Id: US-Twt
ws: WS rs: SW_IN tmean: TA rh: RH_PI_F
daily ameriflux df (3016, 13)
merged df (660, 18)
-----------------------------------------------------------------
Site Id: US-Bi2
ws: WS rs: SW_IN tmean: TA rh: RH
daily ameriflux df (730, 13)
merged df (227, 18)
-----------------------------------------------------------------
Site Id: US-Myb
No rs
ws: WS rs: [] tmean: TA rh: RH
daily ameriflux df (3516, 13)
merged df (1070, 18)
---------------

In [10]:
full_path = os.path.join(base_path, "Ameriflux/Generated/v1/Ameriflux_EEflux_refET")
df = Helpers.load_data(full_path)  
df.head()

class AmerifluxErrorGraph:
    def __init__(self, path):
        print("AmerifluxErrorGraph Initializer")
        self.path = path
          
    def read_data(self):
        file_name = os.path.join(base_path, self.path)
        df = pd.read_csv(file_name, index_col=None, header=0)
        return df
    

    def get_error_metrics(self, y_true, y_predicted):
        r2_Score = r2_score(y_true, y_predicted)
        rmse_score = np.sqrt(mean_squared_error(y_true, y_predicted))
        mse_score = mean_squared_error(y_true, y_predicted)
        mae_score = mean_absolute_error(y_true, y_predicted)

        def mean_absolute_percentage_error(y_true, y_pred):
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        mape_score = mean_absolute_percentage_error(y_true, y_predicted)
        num = 2
        return (round(r2_Score, num), round(rmse_score, num), round(mse_score, num), 
                round(mae_score, num), round(mape_score, num))


    def generate_errors(self, df, first_col, second_col):
        df = df.replace(to_replace = np.nan, value =0) 
        errors = self.get_error_metrics(df[first_col], df[second_col])
        return errors
    
    def plot_ref_et(self, df, site_id, first_column, second_column, path_to_save):
        fig, ax = plt.subplots(figsize=(40, 17))
        fig.subplots_adjust(bottom=0.15, left=0.2)
        ax.plot(df["Date"], df[first_column], marker='o', markersize=7.0, color='green', label="Ameriflux")
        ax.plot(df["Date"], df[second_column], marker='^', markersize=7.0, color='red', label="EEflux")
        title = site_id + ": Comparison between " + first_column + " and " + second_column
        plt.title(title)
        plt.xticks(rotation=90)
        ax.legend()
#         plt.show()
        full_path = os.path.join(base_path, path_to_save)
        plt.savefig(full_path + site_id + "_etr.png")
        plt.close(fig)

    def prepare_data(self, df):
        unique_sites = df["Site Id"].unique()
        unique_sites

        r2_list, rmse_list, mse_list, mae_list, mape_list = [], [], [], [], []
        first_column = "etr"
        second_column = "ETo EEflux"
        for i in range(len(unique_sites)):
            site_id = unique_sites[i]
            df_site = df[df["Site Id"] == site_id]
            (r2, rmse, mse, mae, mape) = self.generate_errors(df_site, first_column, second_column)
            r2_list.append(r2)
            rmse_list.append(rmse)
            mse_list.append(mse)
            mae_list.append(mae)
            mape_list.append(mape)
            min_value = 1
            max_value = 15
            df_filtered_site = df_site[df_site["etr"].between(left=min_value, right=max_value) &
                                       df_site["ETo EEflux"].between(left=min_value, right=max_value)]
            
            df_filtered_site['Date'] = pd.to_datetime(df_filtered_site["Date"])
            df_filtered_site.sort_values(by="Date", inplace=True, ascending=True)
            #Plot the output feature
            if len(df_filtered_site) > 0:
                self.plot_ref_et(df_filtered_site, site_id, first_column, second_column, "Ameriflux/Generated/v1/Graphs/")

        df_errors = pd.DataFrame({"Site Id": unique_sites,
                                  "True Value": first_column,
                                  "Predicted Value": second_column,
                                  "R2": r2_list,
                                  "RMSE": rmse_list, 
                                  "MSE": rmse_list,
                                  "MAE": mae_list, 
                                  "MAPE": mape_list})
        print(df_errors)
        errors_path = "Ameriflux/Generated/v1/Errors_refET"
        Helpers.export_data(df_errors, errors_path)



full_path = "Ameriflux/Generated/v1/Ameriflux_EEflux_refET.csv"
am_eg = AmerifluxErrorGraph(full_path)
df = am_eg.read_data()
am_eg.prepare_data(df)

AmerifluxErrorGraph Initializer
   Site Id True Value Predicted Value    R2   RMSE    MSE   MAE  MAPE
0   US-Tw1        etr      ETo EEflux -0.11   5.92   5.92  4.58   inf
1   US-Skr        etr      ETo EEflux  0.00   4.89   4.89  4.62   NaN
2   US-Snd        etr      ETo EEflux -0.52   8.15   8.15  6.86   inf
3   US-Twt        etr      ETo EEflux -0.39   7.78   7.78  6.34   NaN
4   US-Bi2        etr      ETo EEflux -0.08   7.52   7.52  6.14   inf
5   US-Myb        etr      ETo EEflux  0.05   3.32   3.32  2.14   NaN
6   US-FR2        etr      ETo EEflux -0.57   7.15   7.15  6.18   inf
7   US-MMS        etr      ETo EEflux  0.49   1.86   1.86  1.37   inf
8   US-Pon        etr      ETo EEflux  0.07   6.61   6.61  5.41   inf
9   US-Me2        etr      ETo EEflux  0.02   6.20   6.20  4.73   NaN
10  US-Ced        etr      ETo EEflux  0.00   4.71   4.71  3.75   NaN
11  US-Kon        etr      ETo EEflux  0.02   7.18   7.18  5.81   inf
12  US-KFS        etr      ETo EEflux -0.04   6.61   6.61 